# Filtering Microcap Alts

In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep


# Functions for pulling from Coinmarketcap
- coindfpull: pulls coin information from cmc including total coin supply
- tokendfpull: pulls token information from cmc including total token supply
- coincircdf: pulls coin circulating supply from cmc
- tokencircdf: pulls token circulating supply from cmc

In [2]:
def coindfpull(pagenum):
    site = 'https://coinmarketcap.com/coins/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[9].split('data-btc="')[1].split(' " data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[9].split('data-btc="')[1].split(' " data-usd="')[0]]
            pricesusd = pricesusd + [columns[15].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            pricesbtc = pricesbtc + [columns[15].split('" data-btc="')[1].split('" data-usd="')[0]]
            volumes24husd = volumes24husd + [columns[18].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            volumes24hbtc = volumes24hbtc + [columns[18].split('" data-btc="')[1].split('" data-usd="')[0]]
            totalsupplies = totalsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def tokendfpull(pagenum):
    site = 'https://coinmarketcap.com/tokens/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    platforms = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            platforms = platforms + [columns[0].split('data-platformsymbol="')[1].split('" ')[0]]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[10].split('data-btc="')[1].split(' " ')[1].split('data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[10].split('data-btc="')[1].split(' " ')[0]]
            pricesusd = pricesusd + [columns[16].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            pricesbtc = pricesbtc + [columns[16].split('data-btc="')[1].split('" ')[0]]
            volumes24husd = volumes24husd + [columns[19].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            volumes24hbtc = volumes24hbtc + [columns[19].split('data-btc="')[1].split('" ')[0]]
            totalsupplies = totalsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0].split('" ')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "Platform": platforms,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def coincircdf(pagenum):
    site = 'https://coinmarketcap.com/coins/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

def tokencirdf(pagenum):
    site = 'https://coinmarketcap.com/tokens/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

# Alternative Function for Pulling all Coin/Token Caps

In [17]:
def pullfulldf():
    site = 'https://coinmarketcap.com/all/views/all/'
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")

    table_start = '<table class="table js-summary-table dataTable no-footer" id="currencies-all" style="font-size: 14px; width: 100%;" role="grid">'
    table_end = '</table>'

    table = soup.find_all('table')[0]

    symbol = []
    cname = []
    mcapsusd = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    totalsupplies = []
    availablesupplies = []
    maxsupplies = []
    change1h = []
    change24h = []
    change7d = []
    i = 1
    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if not('class' in columns[0]):
            continue

        cmcname = columns[0].split('"id-')[1].replace('">','')

        apisite = 'https://api.coinmarketcap.com/v1/ticker/' + cmcname + '/'
        response = requests.get(apisite)
        json_data = json.loads(response.text)

        cname = cname + [cmcname]
        symbol = symbol + [json_data[0]['symbol']]
        mcapsusd = mcapsusd + [json_data[0]['market_cap_usd']]
        pricesusd = pricesusd + [json_data[0]['price_usd']]
        pricesbtc = pricesbtc + [json_data[0]['price_btc']]
        volumes24husd = volumes24husd + [json_data[0]['24h_volume_usd']]
        totalsupplies = totalsupplies + [json_data[0]['total_supply']]
        availablesupplies = availablesupplies + [json_data[0]['available_supply']]
        maxsupplies = maxsupplies + [json_data[0]['max_supply']]
        change1h = change1h + [json_data[0]['percent_change_1h']]
        change24h = change24h + [json_data[0]['percent_change_24h']]
        change7d = change7d + [json_data[0]['percent_change_7d']]
        
        print('Number: ' + str(i) + ' Coin: ' + json_data[0]['symbol'])
        sleep(1)
        i += 1

    fulldict = {'CMCname': cname,
                'Symbol': symbol,
                'Mcap': mcapsusd,
                'USDPrice': pricesusd,
                'BTCPrice': pricesbtc,           
                'Vol24h': volumes24husd,           
                'TotalSupply': totalsupplies,           
                'AvailableSupply': availablesupplies,    
                'MaxSupply': maxsupplies,
                'Change1h': change1h,
                'Change24h': change24h,
                'Change7d': change7d}
    fulldf = pd.DataFrame(fulldict,index = cname)
    
    return fulldf


# Creating dataframe for initial filtering
- Keep only marketcaps with less than 100M
- Remove tokens with 24 hour volumes less than 2% of marketcaps
- Remove tokens with circulating supply/total supply with less than 20%

In [10]:
# Creating DF for Coins

def microfilter1(mcapthreshold, volthreshold, supplythreshold):
    pagenmbr = 1
    coinlist = []
    while type(coindfpull(pagenmbr)) != str:
        coinlist = coinlist + [coindfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    coincirclist = []
    while type(coincircdf(pagenmbr)) != str:
        coincirclist = coincirclist + [coincircdf(pagenmbr)]
        pagenmbr += 1

    coinscirc = pd.concat(coincirclist, axis = 0)
    coins = pd.concat(coinlist, axis = 0)
    coins = coins[coins['MktCapUSD'] != '?']
    coins = pd.concat([coins.drop_duplicates(),coinscirc.drop_duplicates()], axis = 1, join = 'inner')

    # Creating DF for Tokens

    pagenmbr = 1
    tokenlist = []
    while type(tokendfpull(pagenmbr)) != str:
        tokenlist = tokenlist + [tokendfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    tokencirclist = []
    while type(tokencirdf(pagenmbr)) != str:
        tokencirclist = tokencirclist + [tokencirdf(pagenmbr)]
        pagenmbr += 1

    tokenscirc = pd.concat(tokencirclist, axis = 0)
    tokens = pd.concat(tokenlist, axis = 0)
    tokens = tokens[tokens['MktCapUSD'] != '?']
    tokens = pd.concat([tokens.drop_duplicates(),tokenscirc.drop_duplicates()], axis = 1, join = 'inner')

    totaldf = pd.concat([coins,tokens], axis = 0)
    totaldf = totaldf[totaldf['CirculatingSupply'] != 'None']
    totaldf = totaldf[totaldf['Vol24USD'] != 'None']

    totaldf['MktCapUSD'] = pd.to_numeric(totaldf['MktCapUSD'])
    totaldf['CirculatingSupply'] = pd.to_numeric(totaldf['CirculatingSupply'])
    totaldf['TotalSupply'] = pd.to_numeric(totaldf['TotalSupply'])
    totaldf['Vol24USD'] = pd.to_numeric(totaldf['Vol24USD'])
    totaldf['VolRatio'] = totaldf['Vol24USD']/totaldf['MktCapUSD']
    totaldf['SupplyRatio'] = totaldf['CirculatingSupply']/totaldf['TotalSupply']
    
    totaldf = totaldf[totaldf['MktCapUSD'] <= mcapthreshold]
    totaldf = totaldf[totaldf['VolRatio'] >= volthreshold]
    totaldf = totaldf[totaldf['SupplyRatio'] >= supplythreshold]
    
    
    return totaldf

In [11]:
def checkannouncement(cmcname):

    site = 'https://coinmarketcap.com/currencies/' + cmcname + '/'

    res = requests.get(site)
    soup = str(bs(res.text, 'lxml'))

    if "Announcement" in soup:
        announcementsite = soup.split('<li><span class="glyphicon glyphicon-bullhorn text-gray" title="Announcement">')[1].split('<a href="')[1].split('" target')[0]
    else:
        announcementsite = "NA"

    if 'bitcointalk.org' in announcementsite:
        return 1, announcementsite
    else:
        return 0, 'NA'
    
def bctcheck(cmcname):

    annsite = checkannouncement(cmcname)[1]
    parsesoup = annsite.split('=')[0] + '=' + annsite.split('=')[1].split('.')[0]
    res = requests.get(parsesoup)
    soup = str(bs(res.text, 'lxml'))
    discussion = soup.split('<td class="middletext"')[1].split('<span class="prevnext">')[0]
    discussion = discussion.split('<div class="nav"')[0]
    if 'All' in discussion.split(parsesoup)[-1]:
        lastdiscuss = discussion.split(parsesoup)[-2]
    else:
        lastdiscuss = discussion.split(parsesoup)[-1]
        
    if '<td align="right" style="padding-right:' in discussion:
        lastpgIndex = lastdiscuss.split('">')[0]
    else:
        lastpgIndex = lastdiscuss.split('">')[0]

    lastpg = parsesoup + lastpgIndex
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    totpages = soup.split('<td class="middletext"')[1].split('[<b>')[1].split('</b>]')[0]

    forumparse = lastpg.split('=')[1]
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    lastmsg = soup.split(forumparse)[1].split('<tbody>')[-1].split('<div class="smalltext">')[-1].split('</div>')[0]
    
    return totpages, lastmsg

In [ ]:
'''
# Using old method to build dataframe, no change fields available

cmcdf = microfilter1(100000000, 0.02, 0.1)

names = list(cmcdf.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        totpgs, last = bctcheck(cmcname)
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcdf,bctdf],axis = 1)
filterdf.to_csv('cmc2.csv')

'''

In [28]:
cmcfull = pullfulldf()

names = list(cmcfull.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        try:
            totpgs, last = bctcheck(cmcname)
        except:
            totpgs, last = (0,0)
        
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcfull,bctdf],axis = 1)

binance = pd.read_csv('binancelist.csv')
binance.index = binance['Name']
del binance['Ticker'], binance['Name']

bittrex = pd.read_excel('bittrexlist.xlsx')
bittrex.index = bittrex['Name']
del bittrex['Ticker'], bittrex['Name']

kucoin = pd.read_csv('kucoinlist.csv')
kucoin.index = kucoin['Name']
del kucoin['Ticker'], kucoin['Name']

cryptopia = pd.read_csv('cryptopialist.csv')
cryptopia.index = cryptopia['Name']
del cryptopia['Ticker'], cryptopia['Name']

fulldf = pd.concat([filterdf,cryptopia,kucoin,binance,bittrex],axis = 1)
fulldf.head()

fulldf.to_csv('cmc.csv')

Number: 1 Coin: BTC
Number: 2 Coin: ETH
Number: 3 Coin: XRP
Number: 4 Coin: BCH
Number: 5 Coin: ADA
Number: 6 Coin: XEM
Number: 7 Coin: LTC
Number: 8 Coin: XLM
Number: 9 Coin: MIOTA
Number: 10 Coin: TRX
Number: 11 Coin: DASH
Number: 12 Coin: NEO
Number: 13 Coin: XMR
Number: 14 Coin: EOS
Number: 15 Coin: ICX
Number: 16 Coin: QTUM
Number: 17 Coin: BTG
Number: 18 Coin: LSK
Number: 19 Coin: XRB
Number: 20 Coin: ETC
Number: 21 Coin: XVG
Number: 22 Coin: SC
Number: 23 Coin: OMG
Number: 24 Coin: BCN
Number: 25 Coin: BCC
Number: 26 Coin: ZEC
Number: 27 Coin: PPT
Number: 28 Coin: STRAT
Number: 29 Coin: BTS
Number: 30 Coin: BNB
Number: 31 Coin: DOGE
Number: 32 Coin: ARDR
Number: 33 Coin: SNT
Number: 34 Coin: DCN
Number: 35 Coin: KCS
Number: 36 Coin: STEEM
Number: 37 Coin: USDT
Number: 38 Coin: WAVES
Number: 39 Coin: VEN
Number: 40 Coin: DRGN
Number: 41 Coin: REP
Number: 42 Coin: DGB
Number: 43 Coin: KMD
Number: 44 Coin: DENT
Number: 45 Coin: GNT
Number: 46 Coin: ZRX
Number: 47 Coin: VERI
Number:

Number: 372 Coin: BSD
Number: 373 Coin: MYB
Number: 374 Coin: WISH
Number: 375 Coin: ABY
Number: 376 Coin: TFL
Number: 377 Coin: XLR
Number: 378 Coin: SEQ
Number: 379 Coin: SPR
Number: 380 Coin: CAT
Number: 381 Coin: WILD
Number: 382 Coin: BITUSD
Number: 383 Coin: ALQO
Number: 384 Coin: PIX
Number: 385 Coin: RMC
Number: 386 Coin: CURE
Number: 387 Coin: XVC
Number: 388 Coin: UFR
Number: 389 Coin: BTM
Number: 390 Coin: XPM
Number: 391 Coin: ZEIT
Number: 392 Coin: CSNO
Number: 393 Coin: QWARK
Number: 394 Coin: FRST
Number: 395 Coin: BCY
Number: 396 Coin: DICE
Number: 397 Coin: OBITS
Number: 398 Coin: FYP
Number: 399 Coin: BWK
Number: 400 Coin: SSS
Number: 401 Coin: SPHR
Number: 402 Coin: CANN
Number: 403 Coin: OXY
Number: 404 Coin: NTRN
Number: 405 Coin: TX
Number: 406 Coin: DOPE
Number: 407 Coin: BRK
Number: 408 Coin: ITNS
Number: 409 Coin: SUMO
Number: 410 Coin: DRP
Number: 411 Coin: UNO
Number: 412 Coin: DOT
Number: 413 Coin: PFR
Number: 414 Coin: CRC
Number: 415 Coin: PTC
Number: 416 

Number: 738 Coin: VLT
Number: 739 Coin: FNC
Number: 740 Coin: PCOIN
Number: 741 Coin: BUCKS
Number: 742 Coin: EVIL
Number: 743 Coin: HAL
Number: 744 Coin: STARS
Number: 745 Coin: ABJ
Number: 746 Coin: STS
Number: 747 Coin: DIX
Number: 748 Coin: TAG
Number: 749 Coin: MNM
Number: 750 Coin: CNO
Number: 751 Coin: GTC
Number: 752 Coin: JET
Number: 753 Coin: YTN
Number: 754 Coin: MARS
Number: 755 Coin: RLT
Number: 756 Coin: KAYI
Number: 757 Coin: MCRN
Number: 758 Coin: BCF
Number: 759 Coin: LTB
Number: 760 Coin: CUBE
Number: 761 Coin: HONEY
Number: 762 Coin: UNY
Number: 763 Coin: BTCR
Number: 764 Coin: AMBER
Number: 765 Coin: TEK
Number: 766 Coin: POST
Number: 767 Coin: NTWK
Number: 768 Coin: SPACE
Number: 769 Coin: ITI
Number: 770 Coin: MONK
Number: 771 Coin: HERO
Number: 772 Coin: LCP
Number: 773 Coin: EOT
Number: 774 Coin: TSE
Number: 775 Coin: BERN
Number: 776 Coin: UNIC
Number: 777 Coin: XGR
Number: 778 Coin: GAP
Number: 779 Coin: SLG
Number: 780 Coin: CCRB
Number: 781 Coin: PR
Number: 

Number: 1096 Coin: QLC
Number: 1097 Coin: TSL
Number: 1098 Coin: MOT
Number: 1099 Coin: HTML
Number: 1100 Coin: CAT
Number: 1101 Coin: SPHTX
Number: 1102 Coin: DEW
Number: 1103 Coin: HPY
Number: 1104 Coin: QBT
Number: 1105 Coin: ACE
Number: 1106 Coin: PRO
Number: 1107 Coin: BCX
Number: 1108 Coin: SHND
Number: 1109 Coin: INF
Number: 1110 Coin: IGNIS
Number: 1111 Coin: SBTC
Number: 1112 Coin: B2X
Number: 1113 Coin: NGC
Number: 1114 Coin: CAPP
Number: 1115 Coin: DIM
Number: 1116 Coin: CLUB
Number: 1117 Coin: XTZ
Number: 1118 Coin: FRGC
Number: 1119 Coin: WC
Number: 1120 Coin: ENT
Number: 1121 Coin: CMS
Number: 1122 Coin: LLT
Number: 1123 Coin: BIG
Number: 1124 Coin: MKR
Number: 1125 Coin: UGT
Number: 1126 Coin: AI
Number: 1127 Coin: FIL
Number: 1128 Coin: TAU
Number: 1129 Coin: SFE
Number: 1130 Coin: VIU
Number: 1131 Coin: BTCA
Number: 1132 Coin: CMS
Number: 1133 Coin: GBG
Number: 1134 Coin: EAG
Number: 1135 Coin: HBC
Number: 1136 Coin: HTML5
Number: 1137 Coin: XID
Number: 1138 Coin: IFC


Name: 61 Alt: kyber-network
Name: 62 Alt: pivx
Name: 63 Alt: qash
Name: 64 Alt: request-network
Name: 65 Alt: wax
Name: 66 Alt: power-ledger
Name: 67 Alt: aeternity
Name: 68 Alt: neblio
Name: 69 Alt: aion
Name: 70 Alt: aelf
Name: 71 Alt: bytom
Name: 72 Alt: substratum
Name: 73 Alt: digitalnote
Name: 74 Alt: rchain
Name: 75 Alt: monacoin
Name: 76 Alt: bitcoindark
Name: 77 Alt: nxt
Name: 78 Alt: enigma-project
Name: 79 Alt: maidsafecoin
Name: 80 Alt: time-new-bank
Name: 81 Alt: syscoin
Name: 82 Alt: byteball
Name: 83 Alt: gas
Name: 84 Alt: chainlink
Name: 85 Alt: medibloc
Name: 86 Alt: santiment
Name: 87 Alt: quantstamp
Name: 88 Alt: tenx
Name: 89 Alt: iconomi
Name: 90 Alt: deepbrain-chain
Name: 91 Alt: digixdao
Name: 92 Alt: gnosis-gno
Name: 93 Alt: zcoin
Name: 94 Alt: red-pulse
Name: 95 Alt: poet
Name: 96 Alt: paccoin
Name: 97 Alt: walton
Name: 98 Alt: civic
Name: 99 Alt: raiden-network-token
Name: 100 Alt: storm
Name: 101 Alt: blockv
Name: 102 Alt: cobinhood
Name: 103 Alt: bancor
Name

Name: 403 Alt: oxycoin
Name: 404 Alt: neutron
Name: 405 Alt: transfercoin
Name: 406 Alt: dopecoin
Name: 407 Alt: breakout
Name: 408 Alt: intensecoin
Name: 409 Alt: sumokoin
Name: 410 Alt: dcorp
Name: 411 Alt: unobtanium
Name: 412 Alt: dotcoin
Name: 413 Alt: payfair
Name: 414 Alt: crowdcoin
Name: 415 Alt: pesetacoin
Name: 416 Alt: leocoin
Name: 417 Alt: buzzcoin
Name: 418 Alt: pluton
Name: 419 Alt: nvo
Name: 420 Alt: monacocoin
Name: 421 Alt: exchange-union
Name: 422 Alt: hyperstake
Name: 423 Alt: privatix
Name: 424 Alt: goldcoin
Name: 425 Alt: synergy
Name: 426 Alt: atmos
Name: 427 Alt: encryptotel
Name: 428 Alt: luxcoin
Name: 429 Alt: 2give
Name: 430 Alt: bitzeny
Name: 431 Alt: gobyte
Name: 432 Alt: memetic
Name: 433 Alt: creditbit
Name: 434 Alt: royal-kingdom-coin
Name: 435 Alt: belacoin
Name: 436 Alt: straks
Name: 437 Alt: bitcoin-plus
Name: 438 Alt: b2bx
Name: 439 Alt: xaurum
Name: 440 Alt: startcoin
Name: 441 Alt: syndicate
Name: 442 Alt: dnotes
Name: 443 Alt: breakout-stake
Name:

Name: 734 Alt: universe
Name: 735 Alt: bitcoin-red
Name: 736 Alt: dashcoin
Name: 737 Alt: ethereum-gold
Name: 738 Alt: veltor
Name: 739 Alt: fincoin
Name: 740 Alt: pioneer-coin
Name: 741 Alt: swagbucks
Name: 742 Alt: evil-coin
Name: 743 Alt: halcyon
Name: 744 Alt: starcash-network
Name: 745 Alt: abjcoin
Name: 746 Alt: stress
Name: 747 Alt: dix-asset
Name: 748 Alt: tagcoin
Name: 749 Alt: mineum
Name: 750 Alt: coin
Name: 751 Alt: global-tour-coin
Name: 752 Alt: jetcoin
Name: 753 Alt: yenten
Name: 754 Alt: marscoin
Name: 755 Alt: roulettetoken
Name: 756 Alt: kayicoin
Name: 757 Alt: macron
Name: 758 Alt: bitcoinfast
Name: 759 Alt: litebar
Name: 760 Alt: digicube
Name: 761 Alt: honey
Name: 762 Alt: unity-ingot
Name: 763 Alt: bitcurrency
Name: 764 Alt: ambercoin
Name: 765 Alt: tekcoin
Name: 766 Alt: postcoin
Name: 767 Alt: network-token
Name: 768 Alt: spacecoin
Name: 769 Alt: iticoin
Name: 770 Alt: monkey-project
Name: 771 Alt: sovereign-hero
Name: 772 Alt: litecoin-plus
Name: 773 Alt: eot-t

Name: 1057 Alt: orlycoin
Name: 1058 Alt: mtmgaming
Name: 1059 Alt: impulsecoin
Name: 1060 Alt: cryptoescudo
Name: 1061 Alt: bowscoin
Name: 1062 Alt: arbit
Name: 1063 Alt: tagrcoin
Name: 1064 Alt: crtcoin
Name: 1065 Alt: osmiumcoin
Name: 1066 Alt: letitride
Name: 1067 Alt: high-voltage
Name: 1068 Alt: ponzicoin
Name: 1069 Alt: qibuck-asset
Name: 1070 Alt: corethum
Name: 1071 Alt: xonecoin
Name: 1072 Alt: ibank
Name: 1073 Alt: biobar
Name: 1074 Alt: frazcoin
Name: 1075 Alt: gbcgoldcoin
Name: 1076 Alt: ccminer
Name: 1077 Alt: sydpak
Name: 1078 Alt: lex4all
Name: 1079 Alt: concoin
Name: 1080 Alt: sojourn
Name: 1081 Alt: ulatech
Name: 1082 Alt: pizzacoin
Name: 1083 Alt: burstocean
Name: 1084 Alt: applecoin-apw
Name: 1085 Alt: appcoins
Name: 1086 Alt: atmcoin
Name: 1087 Alt: nebulas-token
Name: 1088 Alt: genaro-network
Name: 1089 Alt: canyacoin
Name: 1090 Alt: smartmesh
Name: 1091 Alt: medishares
Name: 1092 Alt: game
Name: 1093 Alt: bibox-token
Name: 1094 Alt: bitcoin-diamond
Name: 1095 Alt:

Name: 1375 Alt: president-johnson
Name: 1376 Alt: rhfcoin
Name: 1377 Alt: aidos-kuneen
Name: 1378 Alt: royalties
Name: 1379 Alt: embercoin
Name: 1380 Alt: xylo
Name: 1381 Alt: santa-coin
Name: 1382 Alt: madcoin
Name: 1383 Alt: casinocoin
Name: 1384 Alt: bitcoin-unlimited
Name: 1385 Alt: qcash
Name: 1386 Alt: farstcoin
